In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings

from dotenv import load_dotenv

warnings.filterwarnings("ignore")

In [3]:
from app.utils.data_loader import DataLoader
from app.utils.health_score import HealthScore
from app.utils.portfolios import Portfolios
from app.utils.portfolios_repo import PortfoliosRepository
from app.utils.portfolios_service import PortfolioService
from app.utils.portprop_matrices import PortpropMatrices
from app.utils.portprop_matrices_repo import PortpropMatricesRepository
from app.utils.rebalancer import Rebalancer
from app.utils.rebalancer_repo import RebalancerRepository

## Load Data

In [4]:
load_dotenv()  # Load environment variables from .env file
print(os.getenv("LOAD_DATA_FROM_DWH"))

false


In [5]:
data_loader_dwh = DataLoader(load_from_db=True)
data_loader_parquet = DataLoader(load_from_db=False)

In [6]:
ppm_repo = PortpropMatricesRepository(data_loader=data_loader_parquet)
ports_repo = PortfoliosRepository(data_loader=data_loader_dwh)
rebalancer_repo = RebalancerRepository(data_loader=data_loader_parquet)

In [ ]:
client_out_enriched = ports_repo.load_client_out_product_enriched(
    as_of_date="2025-09-30"
)

[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.
[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.


[DataLoader] Warning: casting dtypes: "Only a column name can be used for the key in a dtype mappings argument. 'port_investment_style' not found in columns."


In [7]:
client_styles = ports_repo.load_client_style(as_of_date="2025-09-30")

[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.


In [21]:
client_styles

,as_of_date,customer_id,port_investment_style,client_tier,business_unit,client_segment_by_inv_aum,client_sub_segment_by_inv_aum,sales_id,ui_client,sales_first_name_en,sales_team
0,2025-09-30,10817,Unwavering,Edge Gold,EDGE,Non-qualified,<2mb,530,N,Financial Solution Center,Financial Solution Center
1,2025-09-30,11033,Unwavering,Edge Platinum,EDGE,Non-qualified,5-10mb,50,N,Kochaporn,Investment Advisor Team 3
2,2025-09-30,11141,Bulletproof,Edge Gold,EDGE,Non-qualified,<2mb,530,N,Financial Solution Center,Financial Solution Center
3,2025-09-30,11317,Conservative,Wealth,PWM,Qualified,Large,253,Y,Sireekarn,Bunchai Group
4,2025-09-30,11858,Bulletproof,-,EDGE,Non-qualified,Dormant,79,N,HOUSEID,HOUSEID
...,...,...,...,...,...,...,...,...,...,...,...
91530,2025-09-30,114043,Aggressive Growth,Edge,EDGE,Qualified,<2mb,84,N,Digital Edge,Digital Edge
91531,2025-09-30,114234,Bulletproof,Edge,EDGE,Qualified,<2mb,84,N,Digital Edge,Digital Edge
91532,2025-09-30,114321,Bulletproof,Edge,EDGE,Qualified,<2mb,84,N,Digital Edge,Digital Edge
91533,2025-09-30,114502,Bulletproof,Edge,EDGE,Qualified,<2mb,84,N,Digital Edge,Digital Edge


In [22]:
ports_ref_table = {
    "product_mapping": ports_repo.load_product_mapping(),
    "product_underlying": ports_repo.load_product_underlying(),
}

[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.
[WARN] Parameter '_user_agent_entry' is deprecated; use 'user_agent_entry' instead. This parameter will be removed in the upcoming releases.


In [23]:
ppm_ref_dict = {
    "portprop_factsheet": ppm_repo.load_portprop_factsheet(),
    "portprop_benchmark": ppm_repo.load_portprop_benchmark(),
    "portprop_ge_mapping": ppm_repo.load_portprop_ge_mapping(),
    "portprop_fallback": ppm_repo.load_portprop_fallback(),
    "portprop_ret_eow": ppm_repo.load_portprop_ret_eow(),
    "advisory_health_score": ppm_repo.load_advisory_health_score(),
}

In [24]:
rb_ref_dict = {
    "es_sell_list": rebalancer_repo.load_es_sell_list(),
    "product_recommendation_rank_raw": rebalancer_repo.load_product_recommendation_rank_raw(),
    "mandate_allocation": rebalancer_repo.load_mandate_candidates(),
}

## Instances

In [25]:
## Portsfolios
ports_all = Portfolios()
ports_all.set_ref_tables(ports_ref_table)
df_out, df_style, port_ids, port_id_mapping = ports_all.create_portfolio_id(
    client_out_enriched, client_styles, column_mapping=["as_of_date", "customer_id"]
)
ports_all.set_portfolio(df_out, df_style, port_ids, port_id_mapping)

## Portfolio Service
port_service = PortfolioService(ports_all)

## Portprop Matrices
ppm = PortpropMatrices(ppm_ref_dict)

## Health Score
hs = HealthScore()

## Rebalancer
rb = Rebalancer(
    client_investment_style="Moderate High Risk",
    client_classification="UI",
    discretionary_acceptance=0.2,
    new_money=1_000_000,
    product_whitelist=["KKP", "PTTEP"],
    product_blacklist=["KKP GNP", "K-GSELECTU-A(A)"],
)
rb.set_ref_tables(rb_ref_dict)

## Portfolio (Service)

In [26]:
## get list of all customer ids
print(port_service.get_all_customer_ids()[0:5])

[26527, 25914, 31882, 110343, 24191]


In [50]:
## get single port from customer id
port = port_service.get_client_portfolio(customer_id=12560)

In [51]:
port.df_style.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 30503 to 30503
Data columns (total 11 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   port_id                        1 non-null      int64 
 1   port_investment_style          1 non-null      string
 2   client_tier                    1 non-null      string
 3   business_unit                  1 non-null      string
 4   client_segment_by_inv_aum      1 non-null      string
 5   client_sub_segment_by_inv_aum  1 non-null      string
 6   sales_id                       1 non-null      string
 7   ui_client                      1 non-null      string
 8   sales_first_name_en            1 non-null      string
 9   sales_team                     1 non-null      string
 10  portpop_styles                 1 non-null      object
dtypes: int64(1), object(1), string(9)
memory usage: 96.0+ bytes


In [52]:
## get port outstanding
port.df_out

,customer_id,as_of_date,product_id,src_sharecodes,desk,port_type,currency,product_display_name,product_type_desc,asset_class_name,...,is_coverage,expected_return,es_core_port,es_sell_list,flag_top_pick,flag_tax_saving,value,port_id,asset_class_code,weight
169910,12560,2025-09-30,B00174068,NTL26NA,TRADE,L,THB,NTL26NA,Fixed Income,Fixed Income,...,True,0.022,True,<NA>,Not Top-Pick,<NA>,2.070409e+06,1147,AA_FI,0.012308
169915,12560,2025-09-30,C00052743,SS_2001661253(-2171),TRADE,L,THB,KKP Smart Settlement (Individual),Cash,Cash and Cash Equivalent,...,True,0.040,True,<NA>,Not Top-Pick,<NA>,5.884071e+04,1147,AA_CASH,0.000350
169992,12560,2025-09-30,M00232287,KKP PEQ-UI,TRADE,L,THB,KKP PEQ-UI,Mutual Fund,Alternative,...,True,0.100,True,<NA>,Top-Pick,<NA>,2.457990e+06,1147,AA_ALT,0.014612
170869,12560,2025-09-30,C00020418,Saxo USD Collateral,SAXO,L,USD,Saxo USD Collateral,Cash,Cash and Cash Equivalent,...,True,0.048,True,<NA>,Not Top-Pick,<NA>,1.933657e+06,1147,AA_CASH,0.011495
170872,12560,2025-09-30,C00139113,SS_2012871861,TRADE,L,THB,KKP Smart Settlement (Individual),Cash,Cash and Cash Equivalent,...,True,0.040,True,<NA>,Not Top-Pick,<NA>,3.134743e+06,1147,AA_CASH,0.018635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187461,12560,2025-09-30,B00323843,XPCL287A,TRADE,L,THB,XPCL287A,Fixed Income,Fixed Income,...,True,0.022,True,<NA>,Not Top-Pick,<NA>,1.012251e+06,1147,AA_FI,0.006017
187518,12560,2025-09-30,M00164052,KKP PGE-UH-SSF,TRADE,L,THB,KKP PGE-UH-SSF,Mutual Fund,Global Equity,...,True,0.077,True,<NA>,Top-Pick,SSF,9.799241e+05,1147,AA_GE,0.005825
187522,12560,2025-09-30,M00188509,KKP GC-UI-R,TRADE,L,THB,KKP GC-UI-R,Mutual Fund,Fixed Income,...,True,0.064,True,<NA>,Not Top-Pick,<NA>,2.912865e+06,1147,AA_FI,0.017316
187558,12560,2025-09-30,S00271414,VT:arcx,SAXO,L,USD,Vanguard Total World Stock Index Fund ETF,Listed Securities,Global Equity,...,True,0.077,True,<NA>,Top-Pick,<NA>,7.661268e+06,1147,AA_GE,0.045543


In [53]:
port.product_mapping.columns

Index(['product_id', 'src_sharecodes', 'desk', 'port_type', 'currency',
       'product_display_name', 'product_type_desc', 'asset_class_name',
       'symbol', 'pp_asset_sub_class', 'is_risky_asset', 'coverage_prdtype',
       'is_coverage', 'expected_return', 'es_core_port', 'es_sell_list',
       'flag_top_pick', 'flag_tax_saving'],
      dtype='object')

In [54]:
# get port allocation lookthrough
port.get_portfolio_asset_allocation_lookthrough(ppm)

asset_class,port_id,aa_alt,aa_cash,aa_fi,aa_ge,aa_le
0,1147,0.07976,0.052603,0.387497,0.376096,0.104043


In [55]:
# get model allocation
port.get_model_asset_allocation_lookthrough(ppm)

,port_id,port_investment_style,portpop_styles,aa_alt_model,aa_cash_model,aa_fi_model,aa_ge_model,aa_le_model
0,1147,Moderate High Risk,Medium to Moderate High Risk,0.13,0.06,0.27,0.45,0.05


In [56]:
port.df_style

,port_id,port_investment_style,client_tier,business_unit,client_segment_by_inv_aum,client_sub_segment_by_inv_aum,sales_id,ui_client,sales_first_name_en,sales_team,portpop_styles
30503,1147,Moderate High Risk,Wealth,PWM,Qualified,Large,323,Y,Naratporn,Rungsi Group,Medium to Moderate High Risk


## Healthscore

In [57]:
## get client health score
health_score, health_score_comp = port.get_portfolio_health_score(ppm, hs)

In [58]:
health_score["health_score"].values[0]

np.float64(8.0)

In [59]:
health_score_comp

,port_id,product_id,src_sharecodes,desk,port_type,currency,product_display_name,product_type_desc,asset_class_name,value,...,ge_other,expected_return,volatility,is_bulk_risk,underlying_company,issure_risk_group,coverage_prdtype,score_non_cover_global_stock,score_non_cover_local_stock,score_non_cover_mutual_fund
0,1147,B00174068,NTL26NA,TRADE,L,THB,NTL26NA,Fixed Income,Fixed Income,2.070409e+06,...,NaN,0.000271,4.252574e-06,False,NTL,NaN,N/A,0,0,0
1,1147,C00052743,SS_2001661253(-2171),TRADE,L,THB,KKP Smart Settlement (Individual),Cash,Cash and Cash Equivalent,5.884071e+04,...,NaN,0.000014,-8.499198e-09,False,<NA>,NaN,N/A,0,0,0
2,1147,M00232287,KKP PEQ-UI,TRADE,L,THB,KKP PEQ-UI,Mutual Fund,Alternative,2.457990e+06,...,NaN,0.001461,3.224271e-05,False,<NA>,NaN,MUTUAL_FUND,0,0,0
3,1147,C00020418,Saxo USD Collateral,SAXO,L,USD,Saxo USD Collateral,Cash,Cash and Cash Equivalent,1.933657e+06,...,NaN,0.000552,-3.376532e-06,False,<NA>,NaN,N/A,0,0,0
4,1147,C00139113,SS_2012871861,TRADE,L,THB,KKP Smart Settlement (Individual),Cash,Cash and Cash Equivalent,3.134743e+06,...,NaN,0.000745,-4.527953e-07,False,<NA>,NaN,N/A,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,1147,B00323843,XPCL287A,TRADE,L,THB,XPCL287A,Fixed Income,Fixed Income,1.012251e+06,...,NaN,0.000132,2.079142e-06,False,XPCL,NaN,N/A,0,0,0
67,1147,M00164052,KKP PGE-UH-SSF,TRADE,L,THB,KKP PGE-UH-SSF,Mutual Fund,Global Equity,9.799241e+05,...,0.001053,0.000449,9.569741e-04,False,<NA>,NaN,MUTUAL_FUND,0,0,0
68,1147,M00188509,KKP GC-UI-R,TRADE,L,THB,KKP GC-UI-R,Mutual Fund,Fixed Income,2.912865e+06,...,NaN,0.001108,9.103831e-04,False,<NA>,NaN,MUTUAL_FUND,0,0,0
69,1147,S00271414,VT:arcx,SAXO,L,USD,Vanguard Total World Stock Index Fund ETF,Listed Securities,Global Equity,7.661268e+06,...,0.008234,0.003507,7.481839e-03,False,<NA>,NaN,GLOBAL_STOCK,0,0,0


In [60]:
health_score_comp.columns

Index(['port_id', 'product_id', 'src_sharecodes', 'desk', 'port_type',
       'currency', 'product_display_name', 'product_type_desc',
       'asset_class_name', 'value', 'weight', 'aa_alt', 'aa_cash', 'aa_fi',
       'aa_ge', 'aa_le', 'ge_em', 'ge_eur', 'ge_jp', 'ge_us', 'ge_other',
       'expected_return', 'volatility', 'is_bulk_risk', 'underlying_company',
       'issure_risk_group', 'coverage_prdtype', 'score_non_cover_global_stock',
       'score_non_cover_local_stock', 'score_non_cover_mutual_fund'],
      dtype='object')

## Rebalancer

In [61]:
new_port, recommendations = rb.rebalance(port, ppm, hs)

In [62]:
recommendations

,transaction_no,batch_no,port_id,product_id,src_sharecodes,desk,port_type,currency,product_display_name,product_type_desc,asset_class_name,value,weight,flag,expected_weight,action,amount,flag_msg
0,1,1,1147,CTHB00000000,Cash Proxy THB,TRADE,L,THB,Cash Proxy THB,Cash,Cash and Cash Equivalent,NaN,NaN,new_money,NaN,funding,1000000.0,Add new money to the portfolio.
1,2,2,1147,S00087551,BTSGIF,TRADE,L,THB,BTSGIF,Listed Securities,Alternative,3771880.0,0.022290,not_monitored_product,0.000000,sell,-3771880.0,Reduce or exit to manage product not actively ...
2,3,2,1147,CTHB00000000,Cash Proxy THB,TRADE,L,THB,Cash Proxy THB,Cash,Cash and Cash Equivalent,NaN,NaN,cash_proxy_funding,NaN,funding,3771880.0,Fund transactions through the cash proxy posit...
3,4,3,1147,S00237377,VAYU1,TRADE,L,THB,VAYU1,Listed Securities,Local Equity,15900000.0,0.093959,not_monitored_product,0.000000,sell,-15900000.0,Reduce or exit to manage product not actively ...
4,5,3,1147,CTHB00000000,Cash Proxy THB,TRADE,L,THB,Cash Proxy THB,Cash,Cash and Cash Equivalent,NaN,NaN,cash_proxy_funding,NaN,funding,15900000.0,Fund transactions through the cash proxy posit...
5,6,4,1147,DTHB00000000,KKPBATHB,TRADE,L,THB,NaN,NaN,NaN,0.0,0.000000,discretionary_buy,0.122158,buy,20671880.0,Increase allocation to discretionary product a...
6,7,4,1147,CTHB00000000,Cash Proxy THB,TRADE,L,THB,NaN,NaN,NaN,NaN,NaN,cash_proxy_funding,NaN,funding,-20671880.0,Fund transactions through the cash proxy posit...


In [63]:
new_port.df_out

,port_id,product_id,src_sharecodes,desk,port_type,currency,value,product_display_name,product_type_desc,asset_class_name,...,is_risky_asset,coverage_prdtype,is_coverage,expected_return,es_core_port,es_sell_list,flag_top_pick,flag_tax_saving,asset_class_code,weight
0,1147,B00138028,MINT23PA,TRADE,L,THB,4182228.800,MINT23PA,Fixed Income,Fixed Income,...,True,N/A,True,0.022,True,<NA>,Not Top-Pick,<NA>,AA_FI,0.024714
1,1147,B00138680,TUC282A,TRADE,L,THB,2035357.820,TUC282A,Fixed Income,Fixed Income,...,True,N/A,True,0.022,True,<NA>,Not Top-Pick,<NA>,AA_FI,0.012028
2,1147,B00143744,BGRIM23PA,TRADE,L,THB,2048682.660,BGRIM23PA,Fixed Income,Fixed Income,...,True,N/A,True,0.022,True,<NA>,Not Top-Pick,<NA>,AA_FI,0.012106
3,1147,B00145295,LOTUSS284A,TRADE,L,THB,3195077.370,LOTUSS284A,Fixed Income,Fixed Income,...,True,N/A,True,0.022,True,<NA>,Not Top-Pick,<NA>,AA_FI,0.018881
4,1147,B00151378,SIRI25DA,TRADE,L,THB,2009410.860,SIRI25DA,Fixed Income,Fixed Income,...,True,N/A,True,0.040,True,<NA>,Not Top-Pick,<NA>,AA_FI,0.011874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,1147,T00281178,XS3004384742,GIS,L,USD,978282.417,SHFIN20250221A,Structured Note,Global Equity,...,True,N/A,True,0.077,True,<NA>,Not Top-Pick,<NA>,AA_GE,0.005781
66,1147,T00305836,XS2993068878,GIS,L,USD,2619551.144,SHFIN20250401A,Structured Note,Global Equity,...,True,N/A,True,0.077,True,<NA>,Not Top-Pick,<NA>,AA_GE,0.015480
67,1147,T00310048,XS3048279049,GIS,L,USD,1965820.290,SHFIN20250428A,Structured Note,Global Equity,...,True,N/A,True,0.077,True,<NA>,Not Top-Pick,<NA>,AA_GE,0.011617
68,1147,T00320952,XS3102117291,GIS,L,USD,1644450.290,SHFIN20250630A,Structured Note,Global Equity,...,True,N/A,True,0.077,True,<NA>,Not Top-Pick,<NA>,AA_GE,0.009718


In [64]:
health_score, health_score_comp = new_port.get_portfolio_health_score(ppm, hs)

In [65]:
health_score["health_score"].values[0]

np.float64(10.0)